In [59]:
import transformers


In [60]:
nlp.save_pretrained('base_chatbot')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1000}


In [3]:
nlp=transformers.pipeline('conversational',model='base_chatbot')

In [4]:
import pandas as pd
df=pd.read_excel('AttachmentMIT-Academic_Calendar_2023-24.-1 (2).xlsx')

In [5]:
df.drop([0,1],inplace=True)
df.columns=df.iloc[0]

In [6]:
df.drop(2,inplace=True)

In [7]:
df

2,NaN,July -23,Aug -23,Sept -23,Oct -23,Nov -23,Dec -23,NaN,Jan -24,Feb -24,Mar -24,Apr -24,May -24,Jun -24,July -24
3,NaN,1 S,1 T,1 F,1 SUNDAY,1 W Kannada Rajyotsava,1 F,NaN,1M,1 TH,1 F,Last Date to\nApply for\nBranch Change\nUtsav,1 W,1 S,1 M
4,NaN,2 SUNDAY,2 W,2 S,2 M Gandhi Jayanthi,NaN,Publishing list of\nS unsuccessful\nstudents i...,NaN,2T,NaN,NaN,NaN,Publishing list of\n2 TH unsuccessful\nstudent...,NaN,2 T
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 F,Applying for Branch\nChange starts,NaN,NaN,2 SUNDAY,NaN
6,NaN,NaN,NaN,NaN,NaN,2 TH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,3 M,3 TH,3 SUNDAY,NaN,NaN,NaN,NaN,"II, IV, VI Semester\nB. Tech./II Semester\nM.T...",NaN,NaN,NaN,NaN,NaN,3 W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,NaN,NaN,30S Mid-term Exam,NaN,NaN,NaN,NaN,30 T,NaN,30 S,NaN,30 TH,NaN,NaN
129,NaN,NaN,NaN,NaN,30M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 T
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31 W,29 TH,NaN,30T,31 F,30 S,NaN
131,NaN,NaN,NaN,NaN,31 T,30TH,31SUNDAY,NaN,NaN,NaN,31 SUNDAY,NaN,NaN,NaN,31 W


In [11]:
import numpy as np

In [ ]:
df2=pd.read_excel('chat_data.xlsx')

In [ ]:
from datasets import load_dataset

In [ ]:
raw_dataset = load_dataset('csv', data_files = 'chat_data.csv')

In [ ]:
split = raw_dataset['train'].train_test_split(test_size=0.3, seed=42)

In [ ]:
from transformers import AutoTokenizer
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_fn(batch):
    return tokenizer(batch['questions'], truncation = True)

In [ ]:
tokenized_dataset = split.map(tokenize_fn, batched = True)

In [ ]:
vocab_size=20000
maxlen=200

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

In [ ]:
training_args = TrainingArguments(output_dir='training_dir',
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=10,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64,
                                  )

In [ ]:
from sklearn.metrics import f1_score,accuracy_score

In [ ]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    acc = np.mean(predictions == labels)
    f1 = f1_score(labels, predictions, average = 'micro')
    return {'accuracy': acc, 'f1_score': f1}

In [ ]:
trainer = Trainer(model,
                  training_args,
                  train_dataset = tokenized_dataset["train"],
                  eval_dataset = tokenized_dataset["test"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [8]:
from transformers import pipeline

In [9]:
ls training_dir

 Volume in drive C is OS
 Volume Serial Number is 54FE-CB04

 Directory of C:\Users\shivansh\Chatbot\training_dir

27-02-2024  22:53    <DIR>          .
27-02-2024  22:53    <DIR>          ..
27-02-2024  22:49    <DIR>          checkpoint-10
27-02-2024  22:50    <DIR>          checkpoint-15
27-02-2024  22:50    <DIR>          checkpoint-20
27-02-2024  22:51    <DIR>          checkpoint-25
27-02-2024  22:51    <DIR>          checkpoint-30
27-02-2024  22:52    <DIR>          checkpoint-35
27-02-2024  22:52    <DIR>          checkpoint-40
27-02-2024  22:52    <DIR>          checkpoint-45
27-02-2024  22:49    <DIR>          checkpoint-5
27-02-2024  22:53    <DIR>          checkpoint-50
27-02-2024  22:45    <DIR>          runs
               0 File(s)              0 bytes
              13 Dir(s)  32,283,123,712 bytes free


In [11]:
saved_model = pipeline('text-classification',
                       model = 'training_dir/checkpoint-50')

In [16]:
predictions = saved_model(split['test']['questions'])

NameError: name 'split' is not defined

In [ ]:
predictions[:10]

In [17]:
def get_label(d):
    return int(d['label'].split('_')[1])
predictions = [get_label(d) for d in predictions]

NameError: name 'predictions' is not defined

In [ ]:
predictions

In [ ]:
print("acc:",accuracy_score(split['test']['label'], predictions))

In [ ]:
print("f1:",f1_score(split['test']['label'], predictions, average = 'macro'))

In [12]:
saved_model('when is the deepavali vacation ')[0]['label']

'LABEL_1'

In [13]:
import string
translator=str.maketrans('','',string.punctuation)

In [14]:
from spellchecker import SpellChecker
spell=SpellChecker()
def spellcheck(txt):
    t=""
    for word in txt.split():
        w=spell.correction(word)
        if(w==None):
               w=word
        t+=w+" "
    return t.strip()

In [15]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shivansh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
def replace(txt):
    txt=txt.replace('midterm','mid term')
    txt=txt.replace('retest','re test')
    txt=txt.replace('make up','makeup')
    txt=txt.replace('vii','seventh')
    txt=txt.replace('iii','third')
    txt=txt.replace('ii','second')
    
    txt=txt.replace(' iv ',' fourth ')
    txt=txt.replace(' v ',' fifth ')
    txt=txt.replace(' vi ',' sixth ')
    
    txt=txt.replace(' i ',' first ')
    txt=txt.replace('diwali','deepavali')
    
    
    return txt

In [157]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
ls=WordNetLemmatizer()
def preprocess_text(txt):
    txt=txt.lower()
    txt=txt.translate(translator)
    txt=txt.replace('\n',' ')
    txt=replace(txt)
    
    stop_words = set(stopwords.words('english'))
    tokens=[x for x in word_tokenize(txt) if x not in stop_words]
    
    t=""
    for i in tokens:
        t+=ls.lemmatize(i)+" "
    t=t.strip()
    return t
        

In [72]:
import transformers

# Load the RoBERTa model
model = transformers.RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
model.save_pretrained('sent_transformer')

In [19]:
saved_model('When mid term exams')[0]['label']

'LABEL_1'

In [80]:
l=[]
for i in df.columns:
    for j in df[i]:
        if j!=None and type(j)!=float:
            l.append(j)

In [159]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [160]:

def calc_sim(t1,t2):
    emb1=tfidf.fit_transform([t1])
    emb2=tfidf.transform([t2])
    sim= cosine_similarity(emb1,emb2)
    return sim

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
nltk.download('words')
from nltk.corpus import words
eng_words=words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\shivansh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [26]:
def check_vocab(txt):
    tokens= word_tokenize(txt)
    for w in tokens:
        if w not in eng_words:
            return (w, False)
    return (w,True)

In [155]:
def direct_search(txt):
    for i in df.select_dtypes(include='object').columns:
            for j in df[i]:
                if type(j)!=float and len(j)>0:
                    j=preprocess_text(j)
                    if txt in j:
                        return f'{i}:{j}'

In [33]:
preprocess_text("When are my mid term exams?")

when are my mid term exams


'mid term exam'

In [153]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def chat():
    while len(range(1)):
        input_text=input('User: ')
        if input_text.lower()=='bye':
            print('GoodBye')
            break
        input_text=input_text.lower().translate(translator)
        #input_text=spellcheck(input_text)
        input_text=preprocess_text(input_text)
        label=saved_model(input_text)[0]['label']
        f=0
        if(label=='LABEL_1'):
            w,b=check_vocab(input_text)
            if  b!= True:
        
                print(direct_search(w))
            else:
                for i in df.select_dtypes(include='object').columns:
                    for j in df[i]:
                        if type(j)!=float and len(j)>0:
                            j=preprocess_text(j)
                            sim=calc_sim(input_text,j)
                            
                            if sim>0.5:
                                print(sim)
                                f=1
                                print(f'{i}:{j}')
                                break
                    if f==1:
                        break

        else:
            print(nlp(transformers.Conversation(input_text), pad_token_id=50256)[1]['content'])


In [158]:
chat()

User: deepavali holiday?
hi
Nov -23:13m deepavali
User: When are my exam results coming out?
[[0.81649658]]
July -23:18 makeup exam result
User: bye
GoodBye


In [96]:
v=tfidf.fit_transform(l)

In [108]:
sim=cosine_similarity(v)

In [112]:
l[96]

'8 SUNDAY'

In [120]:
sim[1:].idxmax()

AttributeError: 'numpy.ndarray' object has no attribute 'idxmax'

In [113]:
l[0]

'* Holiday only for Students Tentative schedule for Mid-term Re-test: 08/11/2023 to 25/11/2023 ~~Subject to change as per MAHE Circular Tentative Schedule for Mid-term Re-test: 08/04/2024 to 25/04/2024'

array([[0.]])

In [121]:
np.array([1,2,3,6]).argmax()

3

In [134]:
l[300]

'Last Date to\nTH Register for\nSummer Lab session'

In [124]:
l[96]

'8 SUNDAY'

In [161]:
calc_sim('any upcoming event?','any upcoming exam?')

array([[0.81649658]])